# Categorización de llamadas
En este cuaderno se utliza el metodo de text-communities para realizar una clasificación no supervisada de los verbatim del nps

## Importar librerias

In [17]:
import json
import copy
import re
from string import punctuation
from string import digits
import pandas as pd
import seaborn as sns
import networkx as nx
from scipy.stats import powerlaw
from gensim.models import KeyedVectors
sns.set()
import pyemd

## Inicializar modelo de word2vec y text communities
Es necesario inicializar un modelo de word2vec, que se pasa como parametro para el modelo de comunidades de tiempo

In [2]:
model =  KeyedVectors.load_word2vec_format('~/voz-del-cliente/data/SBW-vectors-300-min5.txt', binary=False)

## Leer información de las llamadas
Se leen las llamadas, se convierten a un dataframe y se extraen únicamente los textos de cada llamada

In [18]:
datos= pd.read_csv('verbatim.csv', header=0, delimiter=';')

In [4]:
#Eliminamos puntuación y digitos
datos.Razon = datos.Razon.apply(lambda x: re.sub('[^á-úa-z ]+', '', str(x).lower().strip()))
datos = datos[datos.Razon.notnull()&datos.Razon.apply(lambda x: not x == '')]
d_frases_index = datos[['Razon']].reset_index().set_index('Razon').to_dict()['index']

In [5]:
df_detractor = datos[datos.NPS<7]
df_neutro = datos[(datos.NPS>6) & (datos.NPS<9)]
df_promotor = datos[datos.NPS>8]

## Ajuste inicial del modelo
Se hace un ajuste inicial del modelo de comunidades. El treshold inicial del modelo es 0.85. Este proceso se demora un tiempo significativo.

In [45]:
import numpy as np
import networkx as nx
import community
from spellchecker import SpellChecker
from string import punctuation
from string import digits
from nltk.corpus import stopwords
from sklearn.base import BaseEstimator,ClusterMixin
import logging
from collections import deque
from multiprocessing import Pool
import itertools
import os.path
from google.cloud import storage
from io import StringIO 

SyntaxError: invalid syntax (<ipython-input-45-533f73f44f7f>, line 15)

In [8]:
def calc_distance_betweensentences(ij_sentences):
    sentence_i, sentence_j = ij_sentences
    dist = model.wmdistance(sentence_i, sentence_j)
    
    return (sentence_i, sentence_j , dist)

def get_distances_matrix(vectores, vectores2=None, n_threads=4):

    if vectores2 is None:
        vectores2 = vectores
    
    print('''
        ******************************************************
        ****** Starting Parallel Matrix Calculations *********
        ******************************************************
    ''')

       
    comb = itertools.product(vectores, vectores2)
    #comb = [(i, j, vectores[i], vectores[j]) for i, j in comb]
    
    print(' **** Combinaciones Creadas ****')
    
    #results = None

    # No Parallel
    # results = list(map(calc_distance_betweensentences, comb))

    #Parallel
    with Pool(n_threads) as p:
        results = p.map(calc_distance_betweensentences, comb)

    print(' **** Distancias Calculadas ****')
        
    return results

In [ ]:
def blob_exists(bucket_name, filename):
    client = storage.Client('pro-voz-del-cliente')
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(filename)
    return blob.exists()

In [ ]:
def save_file(dataframe, bucket_name, filename):
    string_io = StringIO()
    dataframe.to_csv(string_io, index=False, encoding="utf-8")
    string_io.seek(0)
    client = storage.Client('pro-voz-del-cliente')
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(filename)
    blob.upload_from_string(string_io.getvalue(), content_type='text/csv')
    return '{} saved'.format(filename)

In [8]:
size = 600 # df_detractor.Razon.size
step = 100
bucket_name = 'pro-voz-del-cliente-nps'
for i in range(0, size, step):
    for j in range(0, size, step):
        if i >= j:
            filename = 'edges_S{}_T{}.csv'.format(i, j)
            print('{} already saved'.format(filename))
            if not blob_exists(bucket_name, filename):
                textos_source = df_detractor.Razon.values[i:i+step]
                textos_target = df_detractor.Razon.values[j:j+step]
                distances = get_distances_matrix(textos_source, textos_target, n_threads=25)
                df = pd.DataFrame([(d_frases_index[f1], d_frases_index[f2], d) for f1, f2, d in distances], columns=['Source', 'Target', 'Weigth'])
                save_file(df, bucket_name, filename)

True

In [ ]:
# Se crea un diccionario para tener trazabilidad de los grafos generados con diferentes distancias
diccionario_grafos = {}


            ******************************************************
            ****** Starting Parallel Matrix Calculations *********
            ******************************************************
        


Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, i

## Actualización parámetros del grafo
Esta función permite actualizar el modelo con un nuevo tresh, calcula la distribucuón de los grados de libertad del grafo y ajusta la distribucion a un power law de la cual calcula elparametro

In [ ]:
def actualizar_modelo(tresh):
    generator.set_params(distances_treshold = tresh)
    diccionario_grafos[generator.get_params()['distances_treshold']] = copy.deepcopy(generator.graph)
    nombre_columna = 'comunidad_' + str(tresh).replace('.', '_')
    df[nombre_columna] = generator.fit_predict(texts)
    nx.write_gexf(generator.graph, str(tresh).replace('.', '_')+".gexf")
    degree_sequence = sorted([d for n, d in diccionario_grafos[tresh].degree()], reverse=True)
    fit = powerlaw.fit(degree_sequence)
    alpha = fit[0]
    print("Con el treshold " + str(tresh) + " el parametro de la distribucion es " + str(alpha))
    sns.distplot(degree_sequence)

In [ ]:
#df.query('community5 == 42').apply(lambda x: print(x.Texto+'\n -------------------------------------'), axis=1)

In [ ]:
actualizar_modelo(0.5)